## algorithm design and anlysis-2025 spring  homework 3
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1  

请证明基于比较的排序算法计算复杂度下界为$\Omega(nlogn)$

![figure](./fig/3.1.png)

## 问题 2 

构建二叉搜索树

> 1. 给定一个数组 [48, 33, 49, 47, 42, 46, 32], 构建一个二叉搜索树存储这些数据，请**绘制**所构建的**二叉搜索树**（标明结点的值）。
> 2. 从第1问的二叉搜索树中删除33，请**绘制**删除33后的**二叉搜索树**（标明结点的值）

- 推荐使用graphviz包绘制树的结构

 1、graphviz 配置出错，故采用其他方式
 
        48
       /  \
      33   49
     /  \
    32   47
         /
        42
          \
           46


2

        48
       /  \
      42   49
     /  \
    32   47
          \
           46


## 问题 3 

> 下面是九门课程的开始和结束时间:
>
> [(9:00,12:30), (11:00,14:00), (13:00, 14:30), (9:00,10:30),(13:00, 14:30),(14:00,16:30), (15:00,16:30), (15:00,16:30), (9:00,10:30)]
>
> 请使用贪婪算法为这九门课分配教室，要求在同一天内安排这些课，并且每个教室同一时间只能安排一门课。

**请问最少需要几间教室，罗列出每个教室安排的课程**

idea：

### 算法思路

1. **按上课时间排序**
   将所有课程按照开始时间从早到晚排序。这样我们可以模拟一天中课程一个接一个地“到达”。

2. **维护一个最小堆存储教室可用时间**

   * 堆中每个元素是 `(end_time, room_id)`，表示该教室在 `end_time` 之后才可再开新课。
   * 每当遇到一门新课，如果堆顶的教室在这门课开始前（`heap[0][0] ≤ start_time`）已空出，就弹出堆顶，把这门课安排到这个教室；
   * 否则，需要新开一个教室，把它分配一个新的 `room_id`。
   * 不论是哪种情况，都要把该教室的新结束时间 `(this_course_end, room_id)` 再 push 回堆中。

3. **记录安排**
   我们额外维护一个字典 `schedule`：键是 `room_id`，值是该教室已排课程的列表。每次分配，就往对应列表里 append 这门课。

4. **结果**

   * 最终堆大小就是所需的最少教室数。
   * `schedule` 中每个 `room_id` 对应的列表就是该教室内的课程时间段（按时间顺序自然排好）。

In [2]:
import heapq

def classrooms(courses):

    courses = sorted(courses, key=lambda x: x[0])
    heap = []
    next_room_id = 1
    schedule = {}
    
    for start, end in courses:
        if heap and heap[0][0] <= start:
            avail_time, room_id = heapq.heappop(heap)
        else:
            room_id = next_room_id
            next_room_id += 1
            schedule[room_id] = []
        schedule[room_id].append((start, end))
        heapq.heappush(heap, (end, room_id))
    
    num_rooms = len(schedule)
    return num_rooms, schedule

def to_minutes(t):
    h, m = map(int, t.split(':'))
    return h*60 + m

def to_hhmm(m):
    return f"{m//60:02d}:{m%60:02d}"


raw = [
    ("09:00","12:30"), ("11:00","14:00"), ("13:00","14:30"),
    ("09:00","10:30"), ("13:00","14:30"), ("14:00","16:30"),
    ("15:00","16:30"), ("15:00","16:30"), ("09:00","10:30")
]
courses = [(to_minutes(s), to_minutes(e)) for s,e in raw]

rooms, sched = classrooms(courses)
print(f"最少需要教室数：{rooms}\n")
for rid in sorted(sched):
    print(f"教室 {rid}：")
    for s,e in sched[rid]:
        print(f"  {to_hhmm(s)} – {to_hhmm(e)}")
    print()

最少需要教室数：3

教室 1：
  09:00 – 12:30
  13:00 – 14:30
  15:00 – 16:30

教室 2：
  09:00 – 10:30
  11:00 – 14:00
  14:00 – 16:30

教室 3：
  09:00 – 10:30
  13:00 – 14:30
  15:00 – 16:30



## 问题 4 

> 爬楼梯问题：假设爬楼梯时你每次只能爬一阶或者爬两阶，问爬上n阶的楼梯，你一共有多少种方法

请设计算法完成该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：

### 算法设计思路

1. **定义子问题**
   设 $f(i)$ 为“到达第 $i$ 级台阶”的不同爬法数目。我们最终想要的是 $f(n)$。

2. **推导递推关系**
   要到达第 $i$ 级台阶，最后一步要么是从第 $i-1$ 级走 1 级上来，要么是从第 $i-2$ 级跨 2 级上来。因此

   $$
     f(i) \;=\; f(i-1)\;+\;f(i-2).
   $$

3. **初始条件**

   * $f(0)=1$：站在地面（相当于“第 0 级”）只有 1 种“爬法”（什么都不爬）；
   * $f(1)=1$：要爬到第 1 级，只能一次爬 1 级。

4. **计算顺序**
   从下标 2 开始，依次按上面的递推式计算到 $f(n)$。

5. **优化空间**
   由于每次只用到前两项，可以用两个变量滚动保存，不必开 $O(n)$ 大小的数组，空间 $O(1)$。


* **时间复杂度**：一次从 $2$ 遍历到 $n$，做常数次加法，$O(n)$。

In [3]:
def climb_stairs(n):
    if n <= 1:
        return 1

    f0, f1 = 1, 1

    for i in range(2, n+1):
        f0, f1 = f1, f0 + f1

    return f1

for n in range(0, 11):
    print(f"n={n:2d}, ways={climb_stairs(n)}")

n= 0, ways=1
n= 1, ways=1
n= 2, ways=2
n= 3, ways=3
n= 4, ways=5
n= 5, ways=8
n= 6, ways=13
n= 7, ways=21
n= 8, ways=34
n= 9, ways=55
n=10, ways=89


## 问题 5 

> 0-1背包问题：现在有4块大理石，每个大理石对应的重量和价值使用一个元组表示, 即（重量，价值），4块大理石的重量和价值为：[(5,10), (4,40), (6,30), (3, 50)], 假设你有一辆最大承重为9的小推车，请问使用这个小推车装大理石的最大价值为多少

请设计一个算法解决该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：

### 算法思路

* **DP **
  令

  $$
    dp[j]=
    \begin{cases}
      \text{背包容量恰好为 }j\text{ 时能取得的最大价值},\quad j=0\ldots W.
    \end{cases}
  $$

* **转移方程**
  对于每块石头 $(w,v)$，我们要么不装入，要么装入：

  $$
    dp[j]\;=\;\max\bigl(dp[j],\;dp[j-w]+v\bigr),
    \quad \text{前提是 }j\ge w.
  $$

  为了保证「每块石头只能用一次」，我们要让容量 $j$ 从大到小遍历。

* **初始条件**

  $$
    dp[0]=0,\quad dp[j>0]=0,
  $$

  表示一开始还没放任何石头，价值都是 0。

* **最终答案**
  扫描完所有石头后，$dp[W]$ 就是最大可取价值。

* **时间复杂度**：有 $n=4$ 件物品，容量 $W=9$。外层遍历物品 $O(n)$，内层容量从 $W$ 到 $w_i$ 为 $O(W)$，总 $O(nW)$。


In [4]:
def knapsack_01(items, capacity):
    dp = [0] * (capacity + 1)
    for w, v in items:
        for j in range(capacity, w - 1, -1):
            dp[j] = max(dp[j], dp[j - w] + v)

    return dp[capacity]

stones = [(5, 10), (4, 40), (6, 30), (3, 50)]
max_weight = 9
result = knapsack_01(stones, max_weight)
print(f"推车最大承重 {max_weight}，可装石头的最大总价值为：{result}")

推车最大承重 9，可装石头的最大总价值为：90


## 问题 6 
给定 `n` 个非负整数表示每个宽度为 `1` 的柱子的高度图，计算按此排列的柱子，下雨之后能接多少雨水。



<div style="text-align:center;width:800px;"> <img alt="" src="./fig/hw3q6.png"> </div>

请分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：

### 算法思路（双指针）

1. **定义两端指针**

   * `l` 从左往右扫，初始 `l=0`；
   * `r` 从右往左扫，初始 `r=n-1`。

2. **维护左右最高柱子**

   * `left_max` 记录遍历过程中左侧看到的最高柱子高度；
   * `right_max` 记录遍历过程中右侧看到的最高柱子高度。

3. **移动较低一侧指针**

   * 每次比较 `height[l]` 和 `height[r]`：

     * 如果 `height[l] < height[r]`，说明左边的当前柱子最高能接水的量只取决于 `left_max`：

       * 若 `height[l] ≥ left_max`，更新 `left_max = height[l]`；
       * 否则，`left_max - height[l]` 就是第 `l` 根柱子上能接的水，累加到答案。
       * 然后 `l += 1` 继续。
     * 否则（`height[r] ≤ height[l]`），对称地用 `right_max` 来计算第 `r` 根柱子能接的水，再 `r -= 1`。

4. **结束条件**
   当 `l > r` 时结束，累计值即为答案。

* **时间复杂度**：指针 `l`、`r` 合计只会移动 $n$ 步，一次扫描完成，$O(n)$。

In [5]:
def trap(height):
    l, r = 0, len(height) - 1
    left_max = right_max = 0
    water = 0

    while l <= r:
        if height[l] < height[r]:
            if height[l] > left_max:
                left_max = height[l]
            else:
                water += left_max - height[l]
            l += 1
        else:
            if height[r] > right_max:
                right_max = height[r]
            else:
                water += right_max - height[r]
            r -= 1

    return water

h1 = [0,1,0,2,1,0,1,3,2,1,2,1]
print(trap(h1))  

h2 = [4,2,0,3,2,5]
print(trap(h2))  

6
9


## 问题 7 

**股票投资组合优化**： 假设你是一位投资者，想要在不同的股票中分配你的资金，以最大化你的投资回报。每只股票都有不同的预期收益率和风险。你的目标是选择一些股票，使得总投资金额不超过你的预算，并且预期收益最大化。

在这个情况下，你可以将每只股票视为一个“物品”，其重量为投资金额，价值为预期收益率。然后，你可以使用分级背包问题的方法来选择部分股票，以便在预算内获得最大的预期收益。

以下是一个简化的例子：

假设你有以下三只股票可供选择：

1. 股票 A：投资金额 5000 美元，预期收益率 10%
2. 股票 B：投资金额 3000 美元，预期收益率 8%
3. 股票 C：投资金额 2000 美元，预期收益率 12%

请设计算法找到最优投资方案，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：

### 思路：把股票选股当作 0–1 背包

* **物品**：每只股票
* **重量** $w_i$：投资该股所需金额
* **价值** $v_i$：预期收益（例如“投资金额×收益率”或直接用收益率加权）
* **背包容量** $B$：你的总预算

目标：选一组股票，使得总重量 $\le B$ 且总价值最大。

1. **状态定义**
   用一维数组 `dp[j]` 表示“恰用预算 $j$ 时最大可得的预期收益”。

2. **初始化**

   ```
   dp[0..B] = 0  
   ```

   表示初始没投任何股，收益为 0。

3. **状态转移**
   依次枚举第 $i$ 只股票 $(w_i, v_i)$，为了保证每只股票只能选一次，预算 $j$ 必须**从大到小**遍历：

   ```
   for i in 0..n-1:
     for j in B down to w_i:
       dp[j] = max(dp[j], dp[j - w_i] + v_i)
   ```

4. **回溯选股方案**
   为了知道到底选了哪些股票，可以额外维护一个二维布尔表 `take[i][j]`，标记在处理第 i 只股、预算 j 时是否“取”了它。

   * 当 `dp[j]` 被“投入第 i 只”这一分支更新时，记 `take[i][j] = True`；否则 `take[i][j] = False`。
   * 最后从 `(i=n-1, j=B)` 开始倒推：若 `take[i][j]` 为真，则把第 $i$ 只加入结果，并令 `j -= w_i`；再看 `i-1`，依此类推。

5. **结果**

   * 最优预期收益：`dp[B]`
   * 最优选股列表：通过 `take` 回溯得到的一组索引。

* **时间**：外层 $n$ 支股票，内层预算循环 $B$ 次，总计 $O(nB)$。


In [6]:
def optimize(stocks, budget):
    n = len(stocks)
    dp = [0] * (budget + 1)
    take = [[False] * (budget + 1) for _ in range(n)]

    for i, (w, v) in enumerate(stocks):
        for j in range(budget, w - 1, -1):
            if dp[j - w] + v > dp[j]:
                dp[j] = dp[j - w] + v
                take[i][j] = True

    res_indices = []
    j = budget
    for i in range(n - 1, -1, -1):
        if take[i][j]:
            res_indices.append(i)
            j -= stocks[i][0]

    res_indices.reverse()
    return dp[budget], res_indices

stocks = [
    (5000, 500),
    (3000, 240),
    (2000, 240),
]
B = 7000  # 假设预算是 7000
max_ret, picks = optimize(stocks, B)
print("最大预期收益：", max_ret)
print("选中股票索引：", picks)
for idx in picks:
    print("  股票", idx, "cost,return =", stocks[idx])

最大预期收益： 740
选中股票索引： [0, 2]
  股票 0 cost,return = (5000, 500)
  股票 2 cost,return = (2000, 240)


## 问题 8

给你二叉搜索树的根节点 root ，该树中的恰好两个节点的值被错误地交换。请在不改变其结构的情况下，恢复这棵树 。设计算法解决该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

e.g.:
---
1.

<div style="text-align:center;width:200px;"> <img alt="" src="./fig/hw3q9-1.jpg"> </div>
输入：root = [1,3,null,null,2]， 输出为：[3,1,null,null,2]， 

解释：3 不能是 1 的左孩子，因为 3 > 1 。交换 1 和 3 使二叉搜索树有效。

2.

 <div style="text-align:center;width:200px;"> <img alt="" src="./fig/hw3q9-1.jpg"> </div>
输入： root=[3,1,4,null,null,2]， 输出为：[2,1,4,null,null,3]

解释：2 不能在 3 的右子树中，因为 2 < 3 。交换 2 和 3 使二叉搜索树有效。

idea：


### 算法设计思路

1. **BST 的中序遍历必然是升序**
   正常情况下，对 BST 做中序遍历会产生一个严格升序的节点值序列。若有两个节点值被交换，则这个序列中会出现至多两处“降序”：

2. **定位那两个节点**

   * 在一次中序遍历内，维护 `prev`（前一个访问的节点），以及 `first` 和 `second`：

     * 当发现 `prev.val > curr.val` 时，如果是**第一次**降序，就把 `first = prev`，`second = curr`；
     * 若是**第二次**降序，再把 `second = curr`（替换上一次记录的 `second`）。
   * 这样遍历结束后，`first` 和 `second` 就分别指向那两个被错换的节点。

3. **交换它们的值**
   只需 `first.val, second.val = second.val, first.val`。

整个过程一次中序遍历，时间复杂度为： $O(n)$。

In [7]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

def recover_bst(root):
    first = second = prev = None

    def inorder(node):
        nonlocal first, second, prev
        if not node:
            return
        inorder(node.left)
        if prev and prev.val > node.val:
            if not first:
                first, second = prev, node
            else:
                second = node
        prev = node
        inorder(node.right)

    inorder(root)
    first.val, second.val = second.val, first.val

def serialize(root):
    from collections import deque
    q = deque([root])
    out = []
    while q:
        node = q.popleft()
        if node:
            out.append(node.val)
            q.append(node.left)
            q.append(node.right)
        else:
            out.append(None)
    while out and out[-1] is None:
        out.pop()
    return out

root = TreeNode(1)
root.left = TreeNode(3)
root.left.right = TreeNode(2)

print("恢复前序列化：", serialize(root))  # [1,3,None,None,2]
recover_bst(root)
print("恢复后序列化：", serialize(root))  # [3,1,None,None,2]


root2 = TreeNode(3)
root2.left = TreeNode(1)
root2.right = TreeNode(4)
root2.right.left = TreeNode(2)

print("恢复前序列化：", serialize(root2))  # [3,1,4,None,None,2]
recover_bst(root2)
print("恢复后序列化：", serialize(root2))  # [2,1,4,None,None,3]


恢复前序列化： [1, 3, None, None, 2]
恢复后序列化： [3, 1, None, None, 2]
恢复前序列化： [3, 1, 4, None, None, 2]
恢复后序列化： [2, 1, 4, None, None, 3]


## 问题 9  

给你一个二叉树的根节点 root ，树中每个节点都存放有一个 0 到 9 之间的数字。每条从根节点到叶节点的路径都代表一个数字：

> 例如，从根节点到叶节点的路径 1 -> 2 -> 3 表示数字 123 。

设计一个算法计算从根节点到叶节点生成的所有数字之和 ，分析算法设计思路，计算时间复杂度，并基于python编程实现

叶节点:是指没有子节点的节点。

e.g.:
---
1.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q10-1.jpg"> </div>

 输入：root = [1,2,3], 输出：25

解释：从根到叶子节点路径 1->2 代表数字 12，1->3 代表数字 13，因此和为 12 + 13 = 25 。

2.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q10-2.jpg"> </div>

 输入：root = [4,9,0,5,1], 输出：1026

解释：从根到叶子节点路径 4->9->5 代表数字 495，4->9->1 代表数字 491，4->0 代表数字 40。因此和为 495 + 491 + 40 = 1026 。

idea：

### 算法思路
我们要把二叉树中每条从根到叶的路径看成一个十进制数（把每个节点的数字「拼接」起来），然后把所有这样的数累加。可以用一次前序遍历（DFS）来完成：

1. **递归维护「当前路径值」**

   * 进入一个节点时，把它的值拼到前面的结果后面：
   
2. **遇到叶子节点**

   * 如果当前节点左右孩子都为空，就把 `newVal` 加到全局总和里。
3. **非叶子节点**

   * 继续递归左右子树，分别传入 `newVal`。
4. **总体结果**

   * 遍历完所有节点后，全局累加器就是答案。

* **时间复杂度**：每个节点只会进入递归一次，做 $O(1)$ 的拼接和判断，总计 $O(n)$。

In [8]:
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

def sum_root_to_leaf(root):
    total = 0

    def dfs(node, cur):
        nonlocal total
        if not node:
            return
        cur = cur * 10 + node.val
        if not node.left and not node.right:
            total += cur
            return
        dfs(node.left, cur)
        dfs(node.right, cur)

    dfs(root, 0)
    return total

root1 = TreeNode(1)
root1.left  = TreeNode(2)
root1.right = TreeNode(3)
print(sum_root_to_leaf(root1))  


root2 = TreeNode(4)
root2.left  = TreeNode(9)
root2.right = TreeNode(0)
root2.left.left  = TreeNode(5)
root2.left.right = TreeNode(1)
print(sum_root_to_leaf(root2))  


25
1026


## 问题 10  

给你一个二叉树的根节点 root ， 检查它是否轴对称。

1. 分析算法设计思路，计算时间复杂度，并基于python编程实现
2. \* 设计使用递归和迭代两种方法解决这个问题，计算时间复杂度，并基于python编程实现

e.g.:
---
1.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q11-1.png"> </div>

 输入：root = [1,2,2,3,4,4,3]， 输出：true

解释：二叉树 [1,2,2,3,4,4,3] 对称。

2.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q11-2.png"> </div>

 输入：root = [1,2,2,null,3,null,3]， 输出：false

idea

### 算法思路

* **何谓对称**
  树是对称的，当且仅当它的左子树和右子树互为镜像。
  “镜像”要求：

  1. 两个节点都为空 → 对称；
  2. 都非空且值相等，并且

     * 左子树的左孩子 ⇔ 右子树的右孩子
     * 左子树的右孩子 ⇔ 右子树的左孩子

* **时间复杂度**
  不管是递归还是迭代，我们都要访问每个节点常数次，故时间都是 $O(n)$，$n$ 为节点总数。

In [12]:
from collections import deque
## 一、递归法
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None
def build_tree(vals):
    if not vals:
        return None
    it = iter(vals)
    root = TreeNode(next(it))
    q = deque([root])
    for v in it:
        node = q[0]
        if v is not None:
            child = TreeNode(v)
            q.append(child)
        else:
            child = None

        # 先填左孩子，再填右孩子
        if not node.left:
            node.left = child
        else:
            node.right = child
            q.popleft()

    return root

def is_symmetric(root):
    if not root:
        return True

    def check(a, b):
        if not a and not b:
            return True
        if not a or not b or a.val != b.val:
            return False
        return check(a.left, b.right) and check(a.right, b.left)

    return check(root.left, root.right)

root1 = build_tree([1,2,2,3,4,4,3])
print(is_symmetric(root1))      

root2 = build_tree([1,2,2,None,3,None,3])
print(is_symmetric(root2))      

True
False


* **迭代法时间复杂度**：$O(n)$

In [13]:
def is_symmetric_iter(root):
    if not root:
        return True
    q = deque()
    q.append((root.left, root.right))

    while q:
        a, b = q.popleft()
        if not a and not b:
            continue
        if not a or not b or a.val != b.val:
            return False
        q.append((a.left,  b.right))
        q.append((a.right, b.left))

    return True

root1 = build_tree([1,2,2,3,4,4,3])
print(is_symmetric_iter(root1))  

root2 = build_tree([1,2,2,None,3,None,3])
print(is_symmetric_iter(root2))  

True
False


## 问题 11  

给你一个由 '1'（陆地）和 '0'（水）组成的的二维网格，请你计算网格中岛屿的数量。岛屿总是被水包围，并且每座岛屿只能由水平方向和/或竖直方向上相邻的陆地连接形成。

此外，你可以假设该网格的四条边均被水包围。

分析算法设计思路，计算时间复杂度，并基于python编程实现

e.g.:
---
1.
输入：grid = [

    ["1","1","1","1","0"],

    ["1","1","0","1","0"], 

    ["1","1","0","0","0"], 

    ["0","0","0","0","0"]  

]

输出：1

2.
输入：grid = [

    ["1","1","0","0","0"],

    ["1","1","0","0","0"],

    ["0","0","1","0","0"],

    ["0","0","0","1","1"]

]

输出：3

idea:

**算法思路**

1. **遍历网格**
   依次扫描每个格子 `(i,j)`：

   * 如果是陆地 `'1'`，说明找到了一个新岛，令岛屿计数器 `count += 1`，然后从这里开始「漫水填充」（DFS 或 BFS），把与它连通的所有陆地都标记成水 `'0'`，以免重复计数。
   * 否则跳过已访问过的或本来就是水的格子。

2. **漫水填充**
   从当前陆地格子出发，递归（或入队）访问它上下左右四个方向的格子：

   * 越界或遇到 `'0'` 则返回；
   * 否则把格子置为 `'0'`，继续填充它的四邻。

* **时间复杂度**：每个格子最多进栈/入队、访问一次，漫水填充将它的连通区域全部“淹没”后不会再重复访问，总共 $O(m \times n)$。


In [ ]:

def num_islands(grid):
    if not grid or not grid[0]:
        return 0

    m, n = len(grid), len(grid[0])
    count = 0

    def dfs(i, j):
        if i < 0 or i >= m or j < 0 or j >= n or grid[i][j] != '1':
            return
        grid[i][j] = '0'
        dfs(i-1, j)
        dfs(i+1, j)
        dfs(i, j-1)
        dfs(i, j+1)

    for i in range(m):
        for j in range(n):
            if grid[i][j] == '1':
                count += 1
                dfs(i, j)

    return count

grid1 = [
    ["1","1","1","1","0"],
    ["1","1","0","1","0"],
    ["1","1","0","0","0"],
    ["0","0","0","0","0"]
]
print(num_islands(grid1))  

grid2 = [
    ["1","1","0","0","0"],
    ["1","1","0","0","0"],
    ["0","0","1","0","0"],
    ["0","0","0","1","1"]
]
print(num_islands(grid2))  